# Kaggle Titanic Model


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

tf.enable_eager_execution
print(tf.__version__)

### Preprocess the data into X and Y arrays

In [ ]:
train_X = pd.DataFrame(pd.read_csv("train.csv"), columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch'
                                                            ,'Ticket','Fare','Cabin','Embarked',])

train_Y = train_X['Survived'].values

train_X = train_X.drop(columns=['PassengerId','Survived','Name','Ticket','Cabin', 'Embarked'])

train_X = train_X.values

for i in range(891):
    if train_X[i,1] == 'male':
        train_X[i,1] = 1
    else:
        train_X[i,1] = 0
    if np.isnan(train_X[i,2]) == True:
        train_X[i,2] = 0
        
print(train_X[1:100,:])
print(train_Y[1:100])

### Build and train the Model

In [ ]:
name = "Titanic_Test_"

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x=train_X, y=train_Y, epochs=175, validation_split= 0.2)


#### Evaluate & Validate correct output

In [ ]:
model.evaluate(train_X,train_Y)
#model.predict(train_X)

### Preprocess the Test data to feed to the model

In [ ]:
test_X = pd.DataFrame(pd.read_csv("test.csv"), columns = ['PassengerId','Pclass','Name','Sex','Age','SibSp','Parch'
                                                            ,'Ticket','Fare','Cabin','Embarked',])

#Setup predictions file
predictfile = np.zeros(shape=(418,2))
predictfile[:,0] = test_X['PassengerId']

test_X = test_X.drop(columns=['PassengerId','Name','Ticket','Cabin', 'Embarked'])

test_X = test_X.values

for i in range(418):
    if test_X[i,1] == 'male':
        test_X[i,1] = 1
    else:
        test_X[i,1] = 0
    if np.isnan(test_X[i,2]) == True:
        test_X[i,2] = 0

### Write the predictions into a listed array

In [ ]:
predictions = model.predict(test_X)

for i in range(418):
    if predictions[i] > 0.5:
        predictions[i] = 1
        predictfile[i,1] = predictions[i]
    else:
        predictions[i] = 0
        predictfile[i,1] = predictions[i]

predictfile = predictfile.astype(np.int32)

predictfile

### Convert the predictions array into a csv file

In [ ]:
predictfile = pd.DataFrame(predictfile, columns = ['PassengerId','Survived'])

#Uncomment only to create file
#predictfile.to_csv('Titanic_predictions.csv')